In [73]:
#import libraries
import urllib , json
import pandas as pd
import sqlite3
from sqlite3 import Error
import logging
import os
import pathlib
import glob

In [74]:
#all the companies for which we have to do a databse entry
company_name=['MSFT','ABB','AAL','AAPL','DELL']

In [75]:
#get the list of uRL's from where we have to fetch data 
def getURL(arr):
    urlList=[]
    for company in arr:
        url=f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED\
        &apikey=NQCFKOVGZASY3EZ9&symbol={company}"
        url = url.replace(" ","") #in case there is any space in URL 
        urlList.append(url)
    return urlList

In [76]:
#imports json from the Restapi 
def ImportJson(url):
    response =urllib.request.urlopen(url)
    data=json.loads(response.read()) #converts to python dict 
    return data

In [89]:
#we have used pathlib as it could easily be used in windows and linux  
db_path=pathlib.Path.cwd().joinpath('sqlite','db','consume.db') 

In [78]:
def createConnection(db_path):
    conn=None
    try:
        conn=sqlite3.connect(db_path)
        return conn
    except Error as e:
        print(e)

In [79]:
#create table 
def CreateTable(conn,query):
    try:
        cursor=conn.cursor()
        cursor.execute(query)
    except Error as e:
        print(e)

In [12]:
#query for creating table and call function accordingly 
sql_query= """ CREATE TABLE IF NOT EXISTS shares (
                                company_name TEXT NOT NULL,
                                dates TEXT NOT NULL,
                                Open REAL , 
                                high  REAL ,
                                low  REAL ,
                                close  REAL ,
                                adjusted_close  REAL ,
                                volume REAL ,
                                dividend_amount  REAL ,
                                split_coefficient  REAL); """
conn=createConnection(db_path)
if conn:
    CreateTable(conn,sql_query)
    conn.commit()
else:
    print("Error! cannot create the database connection.")

In [80]:
conn=sqlite3.connect(db_path)

In [81]:
def CreateEntry(conn,company_name):
    urlList=getURL(company_name)
    cursor=conn.cursor()
    for i,url in enumerate(urlList):
        company=company_name[i]
        data=ImportJson(url)
        entries=[]
        for item in data['Time Series (Daily)'].items():
            cateogary=list(item)
            entries.append(cateogary)
        #initialize variables
        dates=0
        Open = 0
        high = 0
        low = 0
        close =0 
        adjusted_close =0 
        volume=0
        dividend_amount = 0
        split_coefficient=0
        
        #insert data rows into shares
        for date, d in entries:
            dates=date
            Open=d['1. open']
            high=d['2. high']
            low=d['3. low']
            close=d['4. close']
            adjusted_closed=d['5. adjusted close']
            volume=d['6. volume']
            dividend_amount=d['7. dividend amount']
            split_coefficient=d['8. split coefficient']

            cursor.execute("INSERT INTO shares VALUES (?,?,?,?,?,?,?,?,?,?)",(company,dates,Open,high,low,
                                                                              close,adjusted_close,volume,
                                                                              dividend_amount,split_coefficient))

    conn.commit()    

In [20]:
#create the entries for 
CreateEntry(conn,company_name)

In [82]:
company='"ABB"'#of your choice

### Fuctions described in Flask

In [86]:
def getfeatures(conn,company,filename):
    cursor=conn.cursor()
    labels=['Open','high','low','close']
    query="SELECT Open ,high , low , close FROM shares WHERE company_name = " + company
    df =pd.read_sql_query(query,conn)
    print(df.head(4))
    df.to_csv(filename+'.csv',columns=labels, index=False)
    result=cursor.execute(query)
    items=[dict(zip([key[0] for key in cursor.description], row)) for row in result]
    #dump the dictinary into python 
    with open(filename +".json", "w") as outfile: 
        json.dump(items, outfile,indent=4)
    print("The files have been saved to your current directory")

In [91]:
conn=sqlite3.connect(db_path)
cursor=conn.cursor()
company='"ABB"'    
query="SELECT Open ,high , low , close FROM shares WHERE company_name = " + company
print(cursor.execute(query).fetchall())

[(34.6, 34.64, 34.45, 34.49), (33.79, 34.3677, 33.72, 34.3), (34.0, 34.085, 33.7, 33.7), (33.65, 33.95, 33.53, 33.84), (33.36, 33.78, 33.3, 33.74), (33.07, 33.15, 32.8, 32.81), (33.68, 33.78, 33.2, 33.45), (34.64, 34.65, 33.95, 34.06), (34.78, 34.915, 34.735, 34.82), (34.54, 34.63, 34.385, 34.63), (34.6, 34.65, 34.45, 34.55), (34.61, 34.735, 34.55, 34.63), (34.86, 34.87, 34.67, 34.73), (35.11, 35.13, 34.89, 35.03), (35.28, 35.28, 35.04, 35.17), (35.0, 35.135, 34.91, 35.1), (34.6, 34.73, 34.4, 34.63), (34.73, 34.92, 34.65, 34.78), (34.84, 34.87, 34.665, 34.71), (34.07, 34.25, 34.04, 34.14), (33.93, 34.2, 33.92, 34.16), (33.61, 33.82, 33.58, 33.77), (33.83, 33.945, 33.6249, 33.67), (33.54, 33.83, 33.5, 33.75), (33.6, 33.73, 33.45, 33.65), (33.14, 33.4, 33.11, 33.33), (32.81, 33.105, 32.575, 32.94), (34.16, 34.17, 33.83, 33.84), (33.86, 33.9653, 33.683, 33.83), (33.85, 34.32, 33.785, 34.22), (32.74, 33.37, 32.73, 33.26), (33.02, 33.175, 32.511, 32.57), (33.1, 33.42, 32.97, 33.38), (33.86,

In [92]:
company='"ABB"'

In [95]:
def getfeatures(company):
    # global conn ,cursor
    conn=sqlite3.connect(db_path)
    cursor=conn.cursor()
    query="SELECT Open ,high , low , close FROM shares WHERE company_name = " + company
    print("DATABE PATH BELOW")
    print(db_path)
    print("! In am inside the function get feaure")
    print(cursor.execute(query).fetchall())
    df =pd.read_sql_query(query,conn)
    print(df.head(4))
    labels=['Open','high','low','close']
    df.to_csv('feature.csv',columns=labels, index=False)
    result=cursor.execute(query)
    items=[dict(zip([key[0] for key in cursor.description], row)) for row in result]
    #dump the dictinary into python 
    with open('feature.json', "w") as outfile: 
        json.dump(items, outfile,indent=4)
    return "The files have been saved to your current directory"
getfeatures(company)

DATABE PATH BELOW
/home/sakshamgupta/assignment/stock_data_skasham/sqlite/db/consume.db
! In am inside the function get feaure
[(34.6, 34.64, 34.45, 34.49), (33.79, 34.3677, 33.72, 34.3), (34.0, 34.085, 33.7, 33.7), (33.65, 33.95, 33.53, 33.84), (33.36, 33.78, 33.3, 33.74), (33.07, 33.15, 32.8, 32.81), (33.68, 33.78, 33.2, 33.45), (34.64, 34.65, 33.95, 34.06), (34.78, 34.915, 34.735, 34.82), (34.54, 34.63, 34.385, 34.63), (34.6, 34.65, 34.45, 34.55), (34.61, 34.735, 34.55, 34.63), (34.86, 34.87, 34.67, 34.73), (35.11, 35.13, 34.89, 35.03), (35.28, 35.28, 35.04, 35.17), (35.0, 35.135, 34.91, 35.1), (34.6, 34.73, 34.4, 34.63), (34.73, 34.92, 34.65, 34.78), (34.84, 34.87, 34.665, 34.71), (34.07, 34.25, 34.04, 34.14), (33.93, 34.2, 33.92, 34.16), (33.61, 33.82, 33.58, 33.77), (33.83, 33.945, 33.6249, 33.67), (33.54, 33.83, 33.5, 33.75), (33.6, 33.73, 33.45, 33.65), (33.14, 33.4, 33.11, 33.33), (32.81, 33.105, 32.575, 32.94), (34.16, 34.17, 33.83, 33.84), (33.86, 33.9653, 33.683, 33.83), (3

'The files have been saved to your current directory'

In [87]:
getfeatures(conn,company,'feature')

    Open     high    low  close
0  34.60  34.6400  34.45  34.49
1  33.79  34.3677  33.72  34.30
2  34.00  34.0850  33.70  33.70
3  33.65  33.9500  33.53  33.84
The files have been saved to your current directory


In [58]:
def getDifference(company,conn):
    query="SELECT dates , close-Open AS 'DIFFERENCE' FROM shares WHERE company_name=" + company
    df=pd.read_sql_query(query,conn)
    return df.head(5)

In [59]:
getDifference(company,conn)

,dates,DIFFERENCE
0,2021-06-25,-0.11
1,2021-06-24,0.51
2,2021-06-23,-0.30
3,2021-06-22,0.19
4,2021-06-21,0.38


In [60]:
def getAverage(conn):
    query="SELECT dates ,avg(close-Open) AS 'AVG' FROM shares GROUP BY dates"
    df=pd.read_sql_query(query,conn)
    return df.head(5)
getAverage(conn)

,dates,AVG
0,2021-02-03,0.740
1,2021-02-04,0.283
2,2021-02-05,-0.200
3,2021-02-08,0.070
4,2021-02-09,0.281


In [62]:
def MaximumCount(conn,company):
    cursor=conn.cursor()
    query="SELECT Open , close FROM shares WHERE company_name=" + company
    res=cursor.execute(query)
    temp=[dict(zip([key[0] for key in cursor.description],row) for row in res)]
    return getMax(temp)

In [63]:
def getMax(temp):
    Max=0
    currMax=0
    for key in temp:
        for op,cl in key.items():
            if op[1]>cl[1]:
                Max=max(Max,currMax)
                currMax=0
            else:
                currMax+=1
    return f"The number of continuous records having a positive diff are {Max}"

In [64]:
MaximumCount(conn,company)

'The number of continuous records are 7'